This is a tool to transform a Doris report to a Hospital vs Quarters report.

##Instructions

1. Upload the doris file in the left panel.
2. When the file is ready, press "play" and the script will run.
3. When the new spreadsheet is ready, you will see the appropriate message.
4. On the left you should see the new file ready to download. If not, refresh the left panel.

In [ ]:
# @title Run this script by pressing the play button { vertical-output: true, display-mode: "form" }
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill, Border, Side

# Load and clean data from .xls file
df0 = pd.read_excel(r"/content/doris_excel_test.xls", engine='xlrd')

# Clean hospital that get monthly report
exempt = ['V17', 'M77', 'Q70']

# Filter the DataFrame to exclude the specified ProjectIdentifier values
df = df0[~df0['ProjectIdentifier'].isin(exempt)]

# Create pivot table
DF = df.pivot(index="UNIT_DISPLAY_NAME", columns="PERIOD", values="VALIDATION_STATUS")

# Reset 'Index_Column' to a regular data column
DF.reset_index(inplace=True)

# Find unique years
unique_years = DF.columns[1:].str.extract(r'(\d{4})')[0].unique()

# Create separate DataFrames for each year
yearly_dfs = {}
for year in unique_years:
    yearly_dfs[year] = DF[['UNIT_DISPLAY_NAME'] + [col for col in DF.columns if year in col]]

# Save these DataFrames as separate sheets in an Excel file
output_file = r"/content/hospital_data_by_year.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for year, year_df in yearly_dfs.items():
        year_df.to_excel(writer, sheet_name=year, index=False)

#print("Separate sheets for each year have been saved in hospital_data_by_year.xlsx")

# Load the created Excel file for conditional formatting
workbook = openpyxl.load_workbook(output_file)

# Create fill objects with the desired background colors
Red_fill = PatternFill(start_color='00FF0000', end_color='00FF0000', fill_type='solid')
Orange_fill = PatternFill(start_color='00FF9900', end_color='00FF9900', fill_type='solid')
Green_fill = PatternFill(start_color='0099CC00', end_color='0099CC00', fill_type='solid')
Yellow_fill = PatternFill(start_color='00FFCC00', end_color='00FFCC00', fill_type='solid')

# Create a border style
thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

# Apply conditional formatting and borders to each sheet in the workbook
for sheet in workbook.sheetnames:
    worksheet = workbook[sheet]
    for row in worksheet.iter_rows(min_row=1, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column):
        for cell in row:
            # Apply borders to all cells
            cell.border = thin_border

            # Apply conditional formatting
            if cell.row > 1:  # Skip the header row for conditional formatting
                if cell.value == 'In Validation':
                    cell.fill = Orange_fill
                elif cell.value in ['Clean (Pending Outcome)', 'Clean (Lock)']:
                    cell.fill = Green_fill
                elif cell.value is None or cell.value == '':
                    cell.fill = Red_fill
                elif cell.value == 'Awaiting Re-submission':
                    cell.fill = Yellow_fill

    # Adjust column widths
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        worksheet.column_dimensions[column].width = adjusted_width

# Save the modified workbook
workbook.save(output_file)
print("The spreadsheet is ready!")
